In [1]:
import networkx as nx

G = nx.DiGraph()

# add nodes to the graph
G.add_node(1, pos=(0, 0))
G.add_node(2, pos=(1, 0))
G.add_node(3, pos=(1, 1))
G.add_node(4, pos=(0, 1))

# add edges to the graph
G.add_edge(1, 2, weight=1)
G.add_edge(2, 3, weight=1)
G.add_edge(3, 4, weight=1)
G.add_edge(4, 1, weight=1)
# G.edges

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [3]:
# import networkx as nx
# from torch_geometric.utils import to_undirected
# from torch_geometric.data import Data
#
# # # Create a directed graph using networkx
# # G = nx.DiGraph()
# # G.add_edge(0, 1)
# # G.add_edge(1, 2)
# # G.add_edge(2, 0)
#
# # Convert the directed graph to a PyTorch Geometric Data object
# edge_index = torch.tensor(list(G.edges)).t().contiguous()
# data = Data(edge_index=edge_index)
#
# # Convert the directed graph to an undirected graph
# G_undirected = to_undirected(data)

In [6]:
from torch_geometric.utils import to_undirected
from torch_geometric.data import Data, DataLoader

# convert the directed graph to an undirected graph
G_undirected = to_undirected(G)
edge_index = torch.tensor(list(G.edges())).t().contiguous()
G_undirected = to_undirected(edge_index=edge_index)
# create a node feature matrix
x = torch.randn(G_undirected.number_of_nodes(), 2)

# create a target node feature matrix
y = torch.randn(G_undirected.number_of_nodes(), 1)

# create a PyTorch Geometric Data object
data = Data(x=x, edge_index=torch.tensor(G_undirected.edges()).T, y=y)

# create a DataLoader object
loader = DataLoader([data], batch_size=1)

# create a GNN model
model = GNN(in_channels=2, hidden_channels=16, out_channels=1)

# define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# train the model
for epoch in range(100):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

KeyError: 0

In [5]:
G.number_of_nodes()

4